## Get Suspicious DNS

In [46]:
import urllib2
import json
import os

api_root = 'http://localhost:8895/app/api'

# getting date from the parent path.
path = os.getcwd().split("/")
date = path[len(path)-1]
url = '{0}/dns/edge/{1}'.format(api_root, date)

content_type = {'Content-Type': 'application/json'}

# get suspicious IPs.
ip_req = urllib2.Request(url, '{"filter":"ip_dst"}', content_type)
response_ip = urllib2.urlopen(ip_req).read()
ips = json.loads(response_ip)

# get suspicious DNS.
dns_req = urllib2.Request(url, '{"filter":"dns_qry_name"}', content_type)
response_dns = urllib2.urlopen(dns_req).read()
dns = json.loads(response_dns)

##Build Interface

In [47]:
10.# build user interface
from IPython.html import widgets
from IPython.display import display, Javascript
#import ipywidgets as widgets



def fill_list(list_control,source,parameter_value):
    options_list = ['--Select--'] 
    options_list.extend([s[parameter_value] for s in source])
    list_control.options = options_list
   
    
# client panel
client_header = widgets.HTML(value="Client IP")
client_select = widgets.Select(height=190, width=120)  
fill_list(client_select,ips['data'],'ip_dst')
client_select.value = "--Select--"
client_box = widgets.Box(width=150, height=200)
client_box.children = [client_header, client_select]

# query panel
query_header = widgets.HTML(value="Query")
query_select = widgets.Select(height=190, width=250) 
fill_list(query_select,dns['data'],'dns_qry_name')

query_box = widgets.Box(width=300, height=210)
query_box.children = [query_header, query_select]

# Actions Panel
actions_header = widgets.HTML(value="&nbsp;")
quick_text = widgets.Text(value='',width=150, placeholder='Quick scoring')
rating_btn = widgets.RadioButtons(description='Rating:', options=['1', '2', '3'], width=90)
assign_btn = widgets.Button(description='Score')
save_btn = widgets.Button(description='Save')
actions_box = widgets.Box(width=300, height=210)
actions_box.children = [actions_header, quick_text, rating_btn, assign_btn,save_btn]

scoring_form = widgets.HBox(width=600, height=200)
scoring_form.children = [client_box,query_box,actions_box]

display(Javascript("$('.widget-area > .widget-subarea > *').remove();"))
display(scoring_form) 


<IPython.core.display.Javascript object>

## Update Suspicious DNS

In [48]:
import csv
import datetime
import subprocess 

def assign_score(b):
    ip_sev = int(rating_btn.selected_label) if not "--Select--" in client_select.value else ""
    dns_sev = int(rating_btn.selected_label) if not "--Select--" in query_select.value else ""    
    mod_date = datetime.datetime.now()
    mod_user = "IPython"
    
    if quick_text.value:
        ip = ""
        dns = quick_text.value
        
        # Loop over every element in query_select widget
        score_values = []
        for query in query_select.options:
            if query.endswith(dns):
                # Matching element, create one row
                score_values.append((ip,query,ip_sev,dns_sev,mod_date,mod_user))
    else:
        ip = client_select.value if not "--Select--" in client_select.value else ""
        dns = query_select.value if not "--Select--" in query_select.value else ""
        score_values = ((ip,dns,ip_sev,dns_sev,mod_date,mod_user),)
    
    csv_type = 'a' if os.path.isfile("score_tmp.csv") else 'w'     
    
    with open('score_tmp.csv',csv_type) as score:
        wr = csv.writer(score, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        
        # Append every row in score_values to file
        for row in score_values:
            wr.writerow(row)
    
    if ip != "--Select--":
        display(Javascript("$(\"option[data-value='" + ip +"']\").remove();"))

    if quick_text.value:
        display(Javascript("$(\"option[data-value$='" + quick_text.value +"']\").remove();"))
    elif dns != "--Select--":
        display(Javascript("$(\"option[data-value='" + dns +"']\").remove();"))
    
    client_select.value = "--Select--"
    query_select.value = "--Select--"
    quick_text.value = ""


def save(b):
    insert_url = "{0}/dns/scoring".format(api_root)
    data = "{0}/score_tmp.csv".format(os.getcwd())
    insert_req = urllib2.Request(insert_url,'{"data":"'+ data +'","date":"'+ date +'"}', content_type)
    response_insert = urllib2.urlopen(insert_req).read()  
    
    display(Javascript('reloadParentData();'))
        
assign_btn.on_click(assign_score)
save_btn.on_click(save)